<a href="https://colab.research.google.com/github/pu-bi/AI-industry-job-experience-for-non-majors/blob/main/3-BERT/BERT%20chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 33.5 MB/s 
     |████████████████████████████████| 7.6 MB 64.9 MB/s 
     |████████████████████████████████| 182 kB 53.7 MB/s 


## 챗봇의 종류
- 인공지능 기반 챗봇
- Rule based 챗봇
- Hybrid 챗봇

In [ ]:
!pip install -q gradio
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tok = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
mod = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


     |████████████████████████████████| 11.6 MB 25.6 MB/s 
     |████████████████████████████████| 84 kB 100 kB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 106 kB 71.3 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 213 kB 71.2 MB/s 
     |████████████████████████████████| 272 kB 57.2 MB/s 
     |████████████████████████████████| 2.3 MB 58.8 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 80 kB 10.2 MB/s 
     |████████████████████████████████| 68 kB 3.9 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 4.1 MB 62.8 MB/s 
     |████████████████████████████████| 856 kB 66.1 MB/s 
     |████████████████████████████████| 593 kB 67.9 MB/s 


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/863M [00:00<?, ?B/s]

In [ ]:
def predict(input, history=[]):
    # tokenize the new input sentence
    new_user_input_ids = tok.encode(input + tok.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1)

    # generate a response 
    history = mod.generate(bot_input_ids, max_length=1000, pad_token_id=tok.eos_token_id).tolist()

    # convert the tokens to text, and then split the responses into lines
    response = tok.decode(history[0]).split("<|endoftext|>")
    response = [(response[i], response[i+1]) for i in range(0, len(response)-1, 2)]  # convert to tuples of list
    return response, history


In [ ]:
import gradio as gr

gr.Interface(fn=predict,
             inputs=["text", "state"],
             outputs=["chatbot", "state"]).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`

Using Embedded Colab Mode (NEW). If you have issues, please use share=True and file an issue at https://github.com/gradio-app/gradio/
Note: opening the browser inspector may crash Embedded Colab Mode.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

(<gradio.routes.App at 0x7f5f9e4945d0>, 'http://127.0.0.1:7860/', None)

# 버트 대화 시스템 형식으로 만들기

예상 질문: 제주도 관광지 추천해주라.

예상 답변: 아르떼 뮤지엄

<br>



실제 질문: 제주도 관광지 추천해주라.

챗봇 답변: <font color = "skyblue">이번 가을, 연인과 함께 아르떼 뮤지엄 방문 어떠세요?
</font>

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# 지난 데이터와 일치해야 한다!
id2label = {0: '활동', 1: '숙소', 2: '맛집', 3: '날씨', 4: '카페', 5: '인사', 6: '관광지추천', 7: '소개'}

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# (주의) 지난 강의에 저장된 경로에서 모델을 불러올 것!
model_address = '/content/drive/MyDrive/가을_중앙정보처리학원/data_model/my_model' # bert 모델 주소 ex)'/content/drive/MyDrive/week_2/finetuning/checkpoint-5378'

model = AutoModelForSequenceClassification.from_pretrained(f'{model_address}', local_files_only=True)
trainer = Trainer(model=model)

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
# inference 코드에서 예측을 최대값 말고 특정 값을 넘는 threshold로 정해주는 경우
text = "안녕하세요"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
outputs = trainer.model(**encoding)

print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.2994, -2.9187, -2.5539, -3.7701, -2.8220,  2.0911, -3.0564, -3.0223]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
logits = outputs.logits
probs = logits.squeeze().cpu()
print(probs)

tensor([-3.2994, -2.9187, -2.5539, -3.7701, -2.8220,  2.0911, -3.0564, -3.0223],
       grad_fn=<SqueezeBackward0>)


In [ ]:
labels_predict = [ i if i > 0 else 0 for i in probs ]

softmax_probs = F.softmax(probs, dim=-1)
print(softmax_probs)

labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]
if softmax_probs[labels_predict] < 0.5:
  print('unpredictable')
else:
  print(f'predictable -> {id2label[labels_predict]}')

tensor([0.0044, 0.0064, 0.0092, 0.0027, 0.0070, 0.9589, 0.0056, 0.0058],
       grad_fn=<SoftmaxBackward0>)
predictable -> 인사


In [ ]:
id2label

{0: '활동', 1: '숙소', 2: '맛집', 3: '날씨', 4: '카페', 5: '인사', 6: '관광지추천', 7: '소개'}

In [ ]:
# print 자리에 다양한 대답을 위한 문장틀 생성
# input - 질문
# output - 의도에 대한 답변

def answer_system(text):
  # 질문 -> 모델의 인텐션 -> 대화 시스템에 따른 
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = logits.squeeze().cpu()
  labels_predict = [ i if i > 0 else 0 for i in probs ]

  softmax_probs = F.softmax(probs, dim=-1)
  #print(softmax_probs)

  labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]
  pred = id2label[labels_predict] # 질문에 모델의 Intention


  if softmax_probs[labels_predict] < 0.5:
    pred = 'unpredictable'

  sentence = make_sentence(pred)
  return sentence

In [ ]:
id2label.values()

dict_values(['활동', '숙소', '맛집', '날씨', '카페', '인사', '관광지추천', '소개'])

In [ ]:
# 관광지추천의 예시
# 다른 intention에 대한 답변도 작성해 보세요!

import random
def make_sentence(intention):
  if intention not in id2label.values(): # threshold 미만이므로 모르겠다는 것
    sentence = "다시 한번 질문해 주십시오. 답변 가능한 질문은 '활동', '숙소', '맛집', '날씨', '카페', '인사', '관광지추천', '소개' 입니다."
    return sentence

  ## 해당 부분을 의도별로 추가 작성 및 수정!
  if intention == "활동":
    sentence = f"{intention}이(가) 궁금하시군요!"

  elif intention == "숙소":
    sentence = f"{intention}이(가) 궁금하시군요!"

  elif intention == "맛집":
    sentence = f"{intention}이(가) 궁금하시군요!"

  elif intention == "날씨":
    sentence = f"{intention}이(가) 궁금하시군요!"

  elif intention == "카페":
    sentence = f"{intention}이(가) 궁금하시군요!"

  elif intention == "인사":
    sentence = f"{intention}이(가) 궁금하시군요!"

  elif intention == "관광지추천":
    weather = ['봄', '여름', '가을', '겨울']
    choice_weather = random.choice(weather)
    attractions = ['아르떼 뮤지엄', '사려니숲길', '어승생악', '수산저수지', '원 앤 온리', '카멜리아 힐', '수월봉']
    choice_attractions = random.choice(attractions)
    sentence = f'이번 {choice_weather}, {choice_attractions} 가 보시는 건 어때요?'

  elif intention == "소개":
    sentence = "소개"
  
  else:
    sentence = "오류가 발생했습니다"

  return sentence

In [ ]:
answer_system('관광지 좀 알려줄래?')

'이번 봄, 아르떼 뮤지엄 가 보시는 건 어때요?'

In [ ]:
while True:
  print('제주도에 관해서 무엇이든 물어보세요!')
  input_text = input()
  sentence = answer_system(input_text)
  print(sentence)
  print("다시 질문하시겠어요? (y or n)")
  add_q = input()
  add_q = add_q.lower()
  if add_q == 'n' or add_q == 'no' or add_q == '아니' or add_q == '아니오' or add_q == '아뇨':
    print("감사합니다, 또 이용해 주세요.")
    break


제주도에 관해서 무엇이든 물어보세요!
관광지
이번 가을, 사려니숲길 가 보시는 건 어때요?
다시 질문하시겠어요? (y or n)
n
감사합니다, 또 이용해 주세요.


## Gradio 데모

우리가 만든 모델을 이용하여 챗봇을 구현하려고 합니다.

앞서 작성한 `answer_system()` 함수를 활용하여,

`[("사용자가 입력한 인풋 텍스트", "챗봇의 답변")]` 값을 리턴하는 함수 "make_chatbot" 을 아래에 만들어보세요.


In [ ]:
def make_chatbot_gradio(text):
  """코드를 작성해 보세요!"""
  return 

In [ ]:
make_chatbot_gradio('관광지 어디 가볼까')

[('관광지 어디 가볼까', '이번 여름, 사려니숲길 가 보시는 건 어때요?')]

In [ ]:
gr.Interface(fn=make_chatbot_gradio,
             inputs=["text"],
             outputs=["chatbot"]).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`

Using Embedded Colab Mode (NEW). If you have issues, please use share=True and file an issue at https://github.com/gradio-app/gradio/
Note: opening the browser inspector may crash Embedded Colab Mode.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

(<gradio.routes.App at 0x7fceccadb610>, 'http://127.0.0.1:7862/', None)

## Flask 웹데모

우리가 만든 모델을 이용하여 web 데모를 구현하려고 합니다.

앞서 작성한 대화 시스템을 활용하여 웹페이지를 만들어 봅시다!

In [ ]:
# 플라스크 라이브러리 및 웹 실행을 위해 필요한 것들을 설치해줍니다.
!pip install flask --quiet
!pip install flask-ngrok --quiet


# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
print("Install completed!")

Completed!
--2022-11-18 12:52:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  54.3MB/s    in 0.2s    

2022-11-18 12:52:33 (54.3 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]



In [ ]:
# extract the downloaded file using the following command 

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [ ]:
# ngrok 사이트 (https://dashboard.ngrok.com/get-started/setup)에서 회원가입을 한뒤
# "Your Authtoken" 탭을 클릭하여 authtoken을 복사하여 붙여넣습니다.
!authtoken=''
!./ngrok authtoken ${authtoken}

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# 플라스크 모듈 import
from flask import Flask
from flask import render_template, request

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def man():
  return """

  <html>
    <head>
      <style>
        #container {
          display: flex;
          justify-content: center;
          align-items: center;
        }
      </style>
    <head>

    <body bgcolor=#F5F5DC id="container">

      <center>

        <h1> 제주 관광 챗봇 </h1><br>
        <h3> 안녕하세요, 무엇을 도와드릴까요? </h3><br>
          <form method="POST", action="/predict">
            <b> 질문을 입력해주세요 :  <input type="text", name='a', placeholder="sentence"> <br><br>
              <input type="submit" , value='입력 완료' >
          </form>
      
      </center>

    </body>
</html>
"""
    # return render_template('home.html')


@app.route('/predict', methods=['POST'])
def home():
    data1 = request.form['a']
    pred = answer_system(data1)

    return f"""
      <html>
    <head>
      <style>
        #container {{
          display: flex;
          justify-content: center;
          align-items: center;
          text-align: left;
          left: 50%;
        }}
      </style>
    <head>

    <body bgcolor=#F5F5DC id="container">

      <center>

        <h1> 제주 관광 챗봇 </h1><br>
        <h4> 추가 질문을 원하시면 새로 문장을 입력해주세요. </h4><br>
          <form method="POST", action="/predict">
            <b> 질문을 입력해주세요 :  <input type="text", name='a', placeholder="추가 질문"> <br><br>
              <input type="submit" , value='입력 완료' >
          </form>
      
      <h2> {pred} </h2>
      </center>

    </body>
</html>
"""

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b48f-35-230-170-116.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 12:54:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 12:54:35] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 12:54:40] "POST /predict HTTP/1.1" 200 -
